In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/spotifyclassifier-b2d573dee4ec.json"

In [ ]:
!pip install google-api-python-client
from googleapiclient.discovery import build
import json
import pandas as pd
gcs_service = build('storage', 'v1')

In [ ]:
f = gcs_service.objects()
list_request = f.list(bucket='audio-features-fullset')
features_list = []
while list_request is not None:
    results = list_request.execute()
    for d in results['items']:
        id = d['name'][d['name'].find('/') + 1:-5]
        request = f.get_media(bucket=d['bucket'], object=d['name'], generation=d['generation'])
        try:  
            response = request.execute()
            if response != '':
                json_resp = json.loads(response)
                try:
                    features_list.append([id, json_resp["super-genre-label"], json_resp["subgenre-label"], json_resp["danceability"], json_resp['energy'],
                                      json_resp['loudness'], json_resp['mode'], json_resp['speechiness'], json_resp['acousticness'],
                                      json_resp['instrumentalness'], json_resp['liveness'], json_resp['valence'],
                                      json_resp['tempo'], json_resp['time_signature'], json_resp['release_date'][:4], json_resp["popularity"], json_resp["avg_pitches"],
                                      json_resp["avg_timbre"]])
                except KeyError:
                  pass
        except Exception as e:
            print(e)
            pass
    list_request = f.list_next(list_request, results)
pandasDF = pd.DataFrame(features_list, columns=['id', 'super-genre', 'subgenre', 'danceability', 'energy',
                                          'loudness', 'mode', 'speechiness', 'acousticness',
                                          'instrumentalness', 'liveness', 'valence',
                                          'tempo', 'time_signature', 'release_date', 'popularity', 'avg_pitches', 'avg_timbre'])
pandasDF